INSERINDO DADOS NO BANCO DE DADOS ATRAVÉS DO PYTHON

In [65]:
#importa o pandas
import pandas as pd
#importa o pyodbc para conectar ao banco de dados
import pyodbc as dbc


In [71]:
#criando a conexão com o banco de dados sqlserver
dados_conexao_sql_server = (
    #especifica o driver de SGBD utilizado
    'Driver={SQL Server};'
    #nome do servidor
    'Server= {DESKTOP-5BB0L9J};'
    #base de dados que sera utilizada
    'DataBase= {ContosoRetailDW};'
)
#cria a variavel que armazena a conexão 
conexao = dbc.connect(dados_conexao_sql_server)
#printa a conexão
print(f'Sucesso na conexão: {conexao}')

Sucesso na conexão: <pyodbc.Connection object at 0x000002391D403C50>


In [63]:

#cria o controlador para a tabela
controlador = 'Sim'
#enquanto for "Sim" o menu e as funções irão continuar rodando 
while controlador == 'Sim':
    print('MENU DE OPÇÕES')
    print('1. Inserir Dados')
    print('2: Atualizar Dados')
    print('3: Deletar Dados')
    print('4: Sair')
    opcao = int(input('Escolha a opção desejada: '))
    # match case para as opções
    match opcao:
        #insert
        case 1:
            #variaveis que serão utilizadas para alimentar as tabelas
            cliente = input('Nome do cliente: ')
            produto = input('Nome do produto: ')
            preco = float(input('Valor do produto: '))
            quantidade = int(input('Quantidade de Produtos: '))
            
            #cria ao cursor
            cursor = conexao.cursor()
            #comando para inserir dados na tabela
            comando_inserir = f'''INSERT INTO Vendas(Cliente, Produto, Preco, Quantidade) VALUES ('{cliente}', '{produto}', {preco}, {quantidade})'''
            #chama a função, e passa todos os comandos desejados, lista_comandas, se torna uma tupla com todos os comandos que será executados e comitados pelo FOR
            cursor.execute(comando_inserir)
            cursor.commit()
            print('Comandos executados com sucesso!')
        #update
        case 2:
            #menu secundario
            print('Escolher qual tipo de atualização irá relizar.')
            print('1: Nome do Cliente')
            print('2: Nome do Produto')
            print('3: Valor do Produto')
            print('4: Quantidade de Produtos')
            
            #escolha do menu
            opcao = int(input('Escolha a opção desejada: '))
            if opcao == 1:
            #atualiza o nome do comprador
                ##opção 1 para inserir dados
                cliente = input('Nome do cliente: ')
                Id_venda = int(input('Defina o Id do Cliente: '))
                cursor = conexao.cursor()
                comando_update = f''' UPDATE Vendas SET Cliente = '{cliente}' WHERE Id_Venda = '{Id_venda}' '''
                cursor.execute(comando_update)
                cursor.commit()
                print('Nome do cliente atualizado com sucesso!')
            #atualiza o nome do produto
            elif opcao == 2:
                 produto = input('Nome do Produto: ')
                 id_venda = int(input('Defina o Id da Venda: '))
                 cursor = conexao.cursor()
                 comando_update = f''' UPDATE Vendas SET Produto = '{produto}' WHERE Id_Venda = '{id_venda}' '''
                 cursor.execute(comando_update)
                 cursor.commit()
                 print('Nome do produto atualizado com sucesso!')
            #atualiza o preço
            elif opcao == 3:
                preco = float(input('Preco do Produto: '))
                id_venda = int(input('Defina o Id da venda: '))
                cursor = conexao.cursor()
                comando_update = f''' UPDATE Vendas SET Preco = '{preco}' WHERE Id_Venda = '{id_venda}' '''
                cursor.execute(comando_update)
                cursor.commit()
                print('Preco Atualizado com Sucesso!')
            #atualiza a quantide de produtos    
            elif opcao == 4:
                Quantidade_produto = int(input('Quantidade de produtos: '))
                id_venda = int(input('Defina o Id da venda: '))
                cursor = conexao.cursor()
                comando_update = f''' UPDATE Vendas SET Quantidade = '{Quantidade_produto}' WHERE Id_Venda = '{id_venda}' '''
                cursor.execute(comando_update)
                cursor.commit()
                print('Quantidade Atualizada com Sucesso!')
        #Delete            
        case 3:
            id_venda = int(input('Defina o ID da venda que será excluido: '))
            cursor = conexao.cursor()
           
            #comando para deletar linhas da tabela com base no ID
            comando_deletar = f''' DELETE FROM Vendas WHERE Id_Venda = '{id_venda}' '''
            #executa o comando
            cursor.execute(comando_deletar)
            #salva o comando
            cursor.commit()
            #exibe que foi sucesso
            
            print(f'Linha com o ID de venda: {id_venda}, excluido com sucesso!')
            #comando para selecionar o maior id ta tabela Vendas
            cursor.execute(''' SELECT MAX(Id_Venda) FROM Vendas ''')
            #variavel recebe o maior id da tabela vendas
            maior_id = cursor.fetchone()[0]
            if maior_id is None:
                #caso a tabela esteja vazia
                print('Não existem mais registros na tabela')
            else:
                #caso haja registros, ira recriar o registro da tabela, para recomeçar na ordem, a partir do maior presente.
                comando_resetar_ids = f''' DBCC CHECKIDENT ('Vendas', RESEED, {maior_id}) ''' 
                #executa a função
                cursor.execute(comando_resetar_ids)
                #salva a função
                cursor.commit()
                #printa para onde foi ajustada a ordem de ids
                print(f'A sequencia de IDs foi ajustada para: {maior_id}')
                           
        case 4:
            #encera o programa
            print('Programa Encerrado. Obrigada por experimentar.')
            #encera o loopíng
            break


    

MENU DE OPÇÕES
1. Inserir Dados
2: Atualizar Dados
3: Deletar Dados
4: Sair


Escolha a opção desejada:  4


Programa Encerrado. Obrigada por experimentar.


LENDO DADOS DE TABELAS DO SQL

In [99]:
#configura o pandas para exibir todos os dataframes com 2 casas decimais
pd.options.display.float_format = "{:,.2f}".format
#cria o cursor
cursor = conexao.cursor()
# define a consulta
tabela_dimProduct = ''' SELECT ColorName, UnitPrice FROM DimProduct '''
#executa a consulta
cursor.execute(tabela_dimProduct)
#fecha o cursor
cursor.close()
#pega a tabela extraida da consulta e transforma em datafamre
df =  pd.read_sql(tabela_dimProduct, conexao)
#realiza as analises,agrupando por cor e retornando, a soma, media, maximo e minimo por cor
df_agrupado = df.groupby('ColorName').agg({'UnitPrice': ['sum','mean','max','min']}).reset_index()
#renomeia as colunas
df_agrupado.rename(columns = {'ColorName': 'Cor', 'sum':'Soma','mean':'Media', 'max':'Maximo', 'min': 'Minimo', 'UnitPrice': 'Preço Unitario'}, inplace = True)
#exibe o dataframe
display(df_agrupado)


C:\Users\brito\AppData\Local\Temp\ipykernel_13148\2610413270.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql(tabela_dimProduct, conexao)


Cor Preço Unitario                       
                          Soma  Media   Maximo Minimo
0         Azure       2,854.80 203.91   290.00 129.00
1         Black     179,839.03 298.74 2,899.99   0.95
2          Blue      83,834.96 425.56 3,199.99   1.99
3         Brown      45,511.61 591.06 3,199.99  29.99
4          Gold      14,379.38 287.59   673.00  13.90
5         Green      49,600.86 670.28 3,199.99   4.99
6          Grey      82,660.06 292.09 3,199.99   0.95
7        Orange      26,472.19 481.31 3,199.99  29.95
8          Pink      23,726.86 282.46 2,652.00   0.95
9        Purple         477.87  79.64   181.00  28.00
10          Red      32,602.42 329.32 2,652.00   4.98
11       Silver     161,587.57 387.50 3,199.99   0.95
12  Silver Grey       5,978.20 427.01   673.00 198.00
13  Transparent           2.94   2.94     2.94   2.94
14        White     185,146.29 366.63 3,199.99   1.99
15       Yellow       3,197.51  88.82   877.50   4.98
16         blue         268.89  89.63   232.00   9.99